### NWM Multi-Request Merger (iterative)

In [10]:
from nwm_reanalysis import NWM
from dask.distributed import Client, LocalCluster
import datetime

In [14]:
import s3fs
import xarray as xr
import dask
import copy

comids = [2043493]
start_date = "2000-01-01"
end_date = "2000-12-31"
_start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
_end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d") + datetime.timedelta(days=1)

request_url = "s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr"
request_variables = ["streamflow", "velocity"]

wb_request_variables = ["inflow", "outflow", "water_sfc_elev"]
wb_request_url = "s3://noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr"

s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=request_url, s3=s3, check=False)

ds = xr.open_zarr(store=store, consolidated=True)
ds

<xarray.Dataset>
Dimensions:     (feature_id: 2776738, time: 367439)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    gage_id     (feature_id) |S15 dask.array<chunksize=(2776738,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [16]:
%%time
#------ Baseline 1 year normal request ----------#

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data_1y = ds[request_variables].sel(feature_id=comids).sel(time=slice(
        f"{_start_date.year}-{_start_date.month}-{_start_date.day}",
        f"{_end_date.year}-{_end_date.month}-{_end_date.day}"
    )).load(optimize_graph=True, traverse=False)
ds_data_1y

CPU times: total: 20.3 s
Wall time: 37.2 s


<xarray.Dataset>
Dimensions:     (time: 8808, feature_id: 1)
Coordinates:
    elevation   (feature_id) float32 347.1
  * feature_id  (feature_id) int32 2043493
    gage_id     (feature_id) |S15 b'               '
    latitude    (feature_id) float32 34.43
    longitude   (feature_id) float32 -83.74
    order       (feature_id) int32 2
  * time        (time) datetime64[ns] 2000-01-01 ... 2001-01-01T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 0.44 0.44 0.44 ... 0.18 0.18 0.18
    velocity    (time, feature_id) float64 0.39 0.39 0.39 ... 0.28 0.28 0.28
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [6]:
%%time
#------ Baseline 3 year normal request ----------#

end_date_3y = "2003-12-31"
_end_date = datetime.datetime.strptime(end_date_3y, "%Y-%m-%d") + datetime.timedelta(days=1)
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data_3y = ds[request_variables].sel(feature_id=comids).sel(time=slice(
        f"{_start_date.year}-{_start_date.month}-{_start_date.day}",
        f"{_end_date.year}-{_end_date.month}-{_end_date.day}"
    )).load(optimize_graph=True, traverse=False)
ds_data_3y

CPU times: total: 2.66 s
Wall time: 7.02 s


<xarray.Dataset>
Dimensions:         (time: 35088, feature_id: 1)
Coordinates:
  * feature_id      (feature_id) int32 10327875
    latitude        (feature_id) float32 40.21
    longitude       (feature_id) float32 -111.8
  * time            (time) datetime64[ns] 2000-01-01 ... 2004-01-01T23:00:00
Data variables:
    inflow          (time, feature_id) float64 17.93 17.93 17.93 ... 16.79 16.8
    outflow         (time, feature_id) float64 88.56 88.55 88.55 ... 83.83 83.83
    water_sfc_elev  (time, feature_id) float32 1.378e+03 1.378e+03 ... 1.378e+03
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id

In [9]:
%%time
#------ 3 year iterative normal request -------#
years = 3
i_dates = []
for y in range(years):
    s_date = copy.copy(_start_date) + datetime.timedelta(days=y*365)
    e_date = copy.copy(_end_date) + datetime.timedelta(days=y*365)
    i_dates.append((s_date, e_date))

ds_data_3y_i = None
for date in i_dates:
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        ds_data_i = ds[request_variables].sel(feature_id=comids).sel(time=slice(
            f"{date[0].year}-{date[0].month}-{date[0].day}",
            f"{date[1].year}-{date[1].month}-{date[1].day}"
        )).load(optimize_graph=True, traverse=False)
    if ds_data_3y_i is None:
        ds_data_3y_i = ds_data_i
    else:
        ds_data_3y_i.merge(ds_data_i)
ds_data_3y_i

    

CPU times: total: 8.09 s
Wall time: 19.4 s


<xarray.Dataset>
Dimensions:         (time: 35088, feature_id: 1)
Coordinates:
  * feature_id      (feature_id) int32 10327875
    latitude        (feature_id) float32 40.21
    longitude       (feature_id) float32 -111.8
  * time            (time) datetime64[ns] 2000-01-01 ... 2004-01-01T23:00:00
Data variables:
    inflow          (time, feature_id) float64 17.93 17.93 17.93 ... 16.79 16.8
    outflow         (time, feature_id) float64 88.56 88.55 88.55 ... 83.83 83.83
    water_sfc_elev  (time, feature_id) float32 1.378e+03 1.378e+03 ... 1.378e+03
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id